### Decision Tree - Label Classification

In [37]:
import sys
sys.path.append('../../../../')

# Import libraries
from utils.score import fold_cross_validate
from utils.print import print_cross_validation_scores
from constant.columns import FEATURES, LABEL, SIZE, TARGET_Y

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [38]:
# Import data
WINDOW_SIZE = f"{SIZE}_{TARGET_Y}" # Windows size = 10 and Target = 5

# Get features with window size
expanded_features = []
for feature in FEATURES:
    for i in range(SIZE):
        expanded_features.append(f'{feature}_{i}')

# Read dataset
df = pd.read_csv(f'./source/{WINDOW_SIZE}/dataset.csv')
X, y = df[expanded_features], df[LABEL]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

#### Cross Validation

In [39]:
# Define the model
clf_cv = DecisionTreeClassifier()
scores = fold_cross_validate(clf_cv, X_test, y_test)

# Print scores
print_cross_validation_scores(scores)

MACRO:
Precision: 0.7204 (0.1464)
Recall: 0.7071 (0.1273)
F1 score: 0.6842 (0.1094)
---------------------------------
MICRO:
Precision: 0.9182 (0.0307)
Recall: 0.9182 (0.0307)
F1 score: 0.9182 (0.0307)
---------------------------------
